In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()

model.features[24] = QuantConv2d(256, 8, kernel_size=3, padding=1)  
model.features[25] = nn.BatchNorm2d(8)
model.features[27] = QuantConv2d(8, 8, kernel_size=3, padding=1)   
model.features[30] = QuantConv2d(8, 512, kernel_size=3, padding=1)  


features = list(model.features)
del features[28]  # Remove the BatchNorm2d layer at index 28
model.features = nn.Sequential(*features)

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.2023, 0.1994, 0.2010])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))
class QuantAwareLoss(nn.Module):
    def __init__(self, base_loss, alpha=0.1):
        super(QuantAwareLoss, self).__init__()
        self.base_loss = base_loss
        self.alpha = alpha

    def forward(self, outputs, targets, model):
        base_loss = self.base_loss(outputs, targets)
        quant_loss = sum([torch.sum(torch.abs(param)) for param in model.parameters() if param.requires_grad])
        return base_loss + self.alpha * quant_loss


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [25, 70, 100, 150]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1  
#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import ReduceLROnPlateau

lr = 0.001
weight_decay = 1e-4
epochs = 500
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(
    model.parameters(),
    lr=lr,  # Initial learning rate
    betas=(0.9, 0.999),  # Default values for beta1 and beta2
    eps=1e-08,  # Small value to prevent division by zero
    weight_decay=weight_decay  # L2 regularization
)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    #adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)
    scheduler.step(prec)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.839 (0.839)	Data 0.200 (0.200)	Loss 2.4246 (2.4246)	Prec 10.156% (10.156%)
Epoch: [0][100/391]	Time 0.046 (0.058)	Data 0.001 (0.006)	Loss 2.0024 (2.1562)	Prec 24.219% (18.796%)
Epoch: [0][200/391]	Time 0.046 (0.052)	Data 0.001 (0.004)	Loss 2.1164 (2.0965)	Prec 20.312% (20.196%)
Epoch: [0][300/391]	Time 0.046 (0.050)	Data 0.001 (0.003)	Loss 2.0287 (2.0633)	Prec 21.875% (20.764%)
Validation starts
Test: [0/79]	Time 0.154 (0.154)	Loss 1.9597 (1.9597)	Prec 16.406% (16.406%)
 * Prec 24.260% 
best acc: 24.260000
Epoch: [1][0/391]	Time 0.238 (0.238)	Data 0.198 (0.198)	Loss 1.8653 (1.8653)	Prec 28.906% (28.906%)
Epoch: [1][100/391]	Time 0.046 (0.048)	Data 0.001 (0.003)	Loss 1.9420 (1.9258)	Prec 24.219% (25.286%)
Epoch: [1][200/391]	Time 0.046 (0.047)	Data 0.001 (0.002)	Loss 1.8767 (1.9185)	Prec 25.781% (25.237%)
Epoch: [1][300/391]	Time 0.046 (0.046)	Data 0.001 (0.002)	Loss 2.0233 (1.9005)	Prec 19.531% (25.859%)
Validation starts
Test: [0/79]	Time 0.136 (0.136)	Loss 1.

KeyboardInterrupt: 

In [3]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8552/10000 (86%)



In [4]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

basic_block0_conv1 = model.features[0] 
basic_block0_conv1.register_forward_pre_hook(save_output)
dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
29 -th layer prehooked
34 -th layer prehooked
38 -th layer prehooked
41 -th layer prehooked
46 -th layer prehooked
50 -th layer prehooked
54 -th layer prehooked


In [5]:
quant_conv_layer = QuantConv2d(in_channels=3, out_channels=16, kernel_size=3)
w_bit = 4
weight_q = model.features[3].weight_q  # Quantized value stored during training
w_alpha = model.features[3].weight_quant.wgt_alpha   # alpha is defined in your model already. bring it out here
w_delta = w_alpha / (2**(w_bit - 1) - 1)  # delta can be calculated by using alpha and w_bit
weight_int = (weight_q / w_delta).int() # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numberslated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]],

         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]],

         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]],

         ...,

         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]],

         [[ 0,  0,  0],
          [ 7,  0,  0],
          [ 0,  0,  0]],

         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]]],


        [[[-7,  0,  7],
          [-7,  0,  7],
          [-7,  0,  7]],

         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]],

         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]],

         ...,

         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]],

         [[ 0, -7, -7],
          [ 7,  0, -7],
          [ 7,  0, -7]],

         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]]],


        [[[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]],

    

In [7]:
save_output.clear()
model.features[3].register_forward_pre_hook(save_output)
sample_data, _ = next(iter(testloader))  # Get a batch of test data
sample_data = sample_data.cuda()
model(sample_data) 

x_bit = 4    
x = save_output.outputs[0][0]  # input of the 2nd conv layer
x_alpha  = quant_conv_layer.act_alpha.data.item()
x_delta = (2 * x_alpha) / (2**x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = (x_q / x_delta).int()
print(x_int) # you should see clean integer numbers 

tensor([[[[ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          ...,
          [-1, -1, -1,  ..., -1, -2, -1],
          [-1, -1, -1,  ..., -2, -1, -2],
          [-1, -1, -1,  ..., -2, -1, -2]],

         [[ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          ...,
          [ 0,  0,  0,  ...,  0, -1,  0],
          [ 0,  0,  0,  ..., -1, -1, -1],
          [ 0,  0,  0,  ..., -1,  0, -1]],

         [[-1, -1, -1,  ..., -1, -1, -1],
          [-1, -1, -1,  ..., -1, -1, -1],
          [-1, -1, -1,  ..., -1, -1, -1],
          ...,
          [ 1,  0,  0,  ...,  0,  0,  0],
          [ 1,  0,  0,  ...,  0,  0,  0],
          [ 1,  0,  0,  ...,  0,  0,  0]]],


        [[[ 2,  2,  2,  ...,  2,  2,  2],
          [ 2,  2,  2,  ...,  2,  2,  2],
          [ 2,  2,  2,  ...,  2,  2,  2],
          ...,
          [ 0, -1, -2,  ...,  1,  1,  1],
        

In [8]:
conv_int = torch.nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, bias=False).cuda()
weight_int_adjusted = weight_int[:, :3, :, :]  # Slice to use only the first 3 input channels
conv_int.weight = torch.nn.parameter.Parameter(weight_int_adjusted.float())
print(x_int.size())
print(weight_int.size())
output_int =  conv_int(x_int.float())    # output_int can be calculated with conv_int and x_int
output_recovered = output_int * x_delta * w_delta  # recover with x_delta and w_delta
print(output_recovered)

torch.Size([128, 3, 32, 32])
torch.Size([64, 64, 3, 3])
tensor([[[[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          ...,
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]],

         [[  0.0000,   3.1631,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   6.3263,   3.1631,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   9.4894,   6.3263,  ...,   0.0000,   0.0000,   0.0000],
          ...,
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,  -6.3263,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ...,  -6.3263,  -6.3263,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ..., -12.652

In [18]:
save_output.clear()
model.features[7].register_forward_pre_hook(save_output)
sample_data, _ = next(iter(testloader))  # Get a batch of test data
sample_data = sample_data.cuda()
model(sample_data) 

hooked_output = save_output.outputs[0][0]

In [22]:
hooked_output = hooked_output[:, :, :30, :30]
output_recovered_reduced = output_recovered.mean(dim=1, keepdim=True)
print("hooked_output size:", hooked_output.size())
print("output_recovered size:", output_recovered.size())
# Compute difference
difference = torch.abs(hooked_output - output_recovered)
print("Mean Difference:", difference.mean())

hooked_output size: torch.Size([128, 3, 30, 30])
output_recovered size: torch.Size([128, 64, 30, 30])


RuntimeError: The size of tensor a (3) must match the size of tensor b (64) at non-singleton dimension 1

In [ ]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_ref.weight = model.features[3].weight_q 

output_ref = conv_ref(x)
print(output_ref)

In [ ]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
weight = model.layer1[0].conv1.weight
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)


In [ ]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model